In [1]:
import sys
sys.path.append("../")
# sys.path.append("../train_model/")

In [2]:
from utils.common_train_utils import *
from utils.draw_candle_image import *
from utils.evaluate_old_models import *
from functools import partial
# from attention_model import CNNAttention
import os
from tabulate import tabulate
import random
import warnings
warnings.filterwarnings("ignore")


2024-12-15 07:10:57.626650: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 07:10:57.648904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 07:10:57.687111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 07:10:57.700072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 07:10:57.734675: I tensorflow/core/platform/cpu_feature_guar

In [3]:
class EvaluateTool1(EvaluateTool):
    def do_before_load_model(self, parent_model_path):
        pass
    
    def load_model_and_transform_function(self, parent_model_path, model_path, config_file_path):
        self.is_load_before = False
        try:
            sys.path.append(f"{parent_model_path}")
            from model_code import CNNAttention
            self.is_load_before = True
            model_loaded: keras.Sequential = keras.models.load_model(model_path, custom_objects={"CNNAttention": CNNAttention})
        except Exception as err:
            print(f"Error: {err}")
            model_loaded: keras.Sequential = keras.models.load_model(model_path)
        
        transform_function = self.get_origin_transform_function(config_file_path)
        return model_loaded, transform_function
    
    
    def do_after_load_model(self, parent_model_path):
        if self.is_load_before:
            sys.path.pop()
        pass

In [4]:
def print_loss_of_all_models_in_dataset_folder(folder_object):
    dataset_path = folder_object.path
    
    config = read_config(dataset_path)
    candle_type_and_directory_save: dict = config["candle_type_and_directory_save"]
    days_result = config["days_result"]
    
    dataset_test = load_dataset_of_each_type_and_combine(f"{dataset_path}/test", candle_type_and_directory_save)
    
    batch_size = random.randint(1, int(len(dataset_test) / 4))
    shuffle_buffer = random.randint(1, int(len(dataset_test) / 3))
    
    dataset_test_1 = dataset_test.shuffle(shuffle_buffer)
    dataset_test_2 = dataset_test_1.batch(batch_size)
    
    # Ví dụ
    all_models_folder_path = os.path.join(os.path.abspath(dataset_path), "model_save")
    evaluate_tool = EvaluateTool1()
    
    df = evaluate_tool.evaluate_all_models_in_dataset_folder(
        all_models_folder_path, 
        dataset_test_2,
        days_result
    )
    
    # print(f"----------------- {folder_name} -----------------")
    # print(df)
    
    df["total_records"] = len(dataset_test)
    df["batch_size"] = batch_size
    df["shuffle_buffer"] = shuffle_buffer
    
    return df

In [5]:
dataset_parent_folder = os.path.abspath("../dataset")

In [6]:
just_symbol = input("symbol = (null is all folder)").lower().strip()

In [7]:
all_dfs = {}
for f in os.scandir(dataset_parent_folder):
    if not f.is_dir(): continue
    if just_symbol and f.name != f"{just_symbol}_with_three_image_ema_macd": continue
    
    df = print_loss_of_all_models_in_dataset_folder(f)
    all_dfs[f.name] = df

I0000 00:00:1734246662.453326   87681 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734246662.579509   87681 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734246662.579569   87681 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734246662.583366   87681 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734246662.583440   87681 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 3.7784 - mae: 1.2809 - mse: 3.7784
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.0372 - mae: 1.3160 - mse: 4.0372


In [11]:
list(all_dfs.keys())

['btc_with_three_image_ema_macd']

In [12]:
for dataset_name in all_dfs:
    print(dataset_name)
    print(tabulate(all_dfs[dataset_name], headers='keys', tablefmt='pretty'))
    print("\n")

btc_with_three_image_ema_macd
+---+-------------------------+--------------------+--------------------+---------------+------------+----------------+
|   |       model_name        |        mse         |        mae         | total_records | batch_size | shuffle_buffer |
+---+-------------------------+--------------------+--------------------+---------------+------------+----------------+
| 0 |  cnn_attention_epoch_2  | 5.5311102867126465 | 1.465242862701416  |      64       |     4      |       10       |
| 1 | cnn_attention_epoch_2_0 | 5.582462310791016  | 1.4571666717529297 |      64       |     4      |       10       |
+---+-------------------------+--------------------+--------------------+---------------+------------+----------------+




In [13]:
# channels_to_time_step_attention_ema_macd_trend_1_1           | 0.9698517322540283 | 0.7524821758270264 |      67       |     9      |       13       |
# # | 7  |      channels_to_time_step_attention_ema_macd_trend_1_1_1_1_1_1_1 